In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
graph_data_path_root = '../../GraphData'
output_root_dir = '../../HetGNN/ProcessedData_HetGCN'

## Node Types

In [103]:
import json
node_type_df = defaultdict(list)
for idx in range(0, 9):
    with open(f'{graph_data_path_root}/process{idx}.jsons', 'r') as fin:
        for line in tqdm(fin.readlines()):
            trace = json.loads(line)
            node_info = trace['node_info']
            for i, vector in enumerate(node_info):
                node_type_df['trace_id'].append(trace['trace_id'])
                node_type_df['node_id'].append(i)
                node_type_df['node_type'].append(vector[4])

node_type_df = pd.DataFrame(node_type_df)
node_type_df

100%|██████████| 14719/14719 [00:06<00:00, 2395.79it/s]


,trace_id,node_id,node_type
0,007efb9578bc4f1ab3eab03cb5188af5.38.1629255886...,0,0
1,007efb9578bc4f1ab3eab03cb5188af5.38.1629255886...,1,2
2,007efb9578bc4f1ab3eab03cb5188af5.38.1629255886...,2,4
3,007efb9578bc4f1ab3eab03cb5188af5.38.1629255886...,3,1
4,007efb9578bc4f1ab3eab03cb5188af5.38.1629255886...,4,1
...,...,...,...
27110858,ffc3e79734424b65942466e0d8d432e2.47.1628705868...,14,4
27110859,ffc3e79734424b65942466e0d8d432e2.47.1628705868...,15,4
27110860,ffc3e79734424b65942466e0d8d432e2.47.1628705868...,16,3
27110861,ffc3e79734424b65942466e0d8d432e2.47.1628705868...,17,4


In [104]:
with open(f'{output_root_dir}/trace_id_to_idx.json', 'r') as fin:
    trace_id_to_idx = json.load(fin)
# trace_id_to_idx

In [105]:
node_type_df['trace_id'] = node_type_df['trace_id'].apply(lambda x: trace_id_to_idx[x])
node_type_df

,trace_id,node_id,node_type
0,0,0,0
1,0,1,2
2,0,2,4
3,0,3,1
4,0,4,1
...,...,...,...
27110858,132484,14,4
27110859,132484,15,4
27110860,132484,16,3
27110861,132484,17,4


In [106]:
node_type_df.groupby(['trace_id', 'node_type'])['node_id'].apply(list).reset_index()

,trace_id,node_type,node_id
0,0,0,"[0, 74, 79, 85, 89, 94, 100, 104, 108, 112, 11..."
1,0,1,"[3, 4, 10, 11, 13, 14, 15, 16, 19, 22, 25, 26,..."
2,0,2,"[1, 75, 80, 86, 90, 95, 101, 105, 109, 113, 11..."
3,0,3,"[5, 7, 12, 17, 18, 20, 21, 23, 24, 27, 30, 31,..."
4,0,4,"[2, 6, 8, 77, 82, 83, 87, 92, 97, 98, 103, 106..."
...,...,...,...
790360,132484,1,[9]
790361,132484,2,[1]
790362,132484,3,[16]
790363,132484,4,"[3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17,..."


In [107]:
node_type_list = node_type_df.groupby(['trace_id', 'node_type'])['node_id'].apply(list) \
    .reset_index().sort_values(['trace_id', 'node_type'])
node_type_list

,trace_id,node_type,node_id
0,0,0,"[0, 74, 79, 85, 89, 94, 100, 104, 108, 112, 11..."
1,0,1,"[3, 4, 10, 11, 13, 14, 15, 16, 19, 22, 25, 26,..."
2,0,2,"[1, 75, 80, 86, 90, 95, 101, 105, 109, 113, 11..."
3,0,3,"[5, 7, 12, 17, 18, 20, 21, 23, 24, 27, 30, 31,..."
4,0,4,"[2, 6, 8, 77, 82, 83, 87, 92, 97, 98, 103, 106..."
...,...,...,...
790360,132484,1,[9]
790361,132484,2,[1]
790362,132484,3,[16]
790363,132484,4,"[3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17,..."


In [111]:
for gid in tqdm(sorted(node_type_list.trace_id.unique())):
    type_list = []
    for i in range(node_type_list.node_type.max() + 1):
        lst_value = node_type_list[(node_type_list.trace_id == gid) & (node_type_list.node_type == i)]['node_id'].values.tolist()

        if len(lst_value) == 0: 
            type_list.append([])
        else:
            type_list.append(lst_value[0])

    with open(f'{output_root_dir}/node_types.txt', 'a') as fout:
        fout.write(json.dumps(type_list))
        fout.write('\n')


100%|██████████| 132485/132485 [30:36<00:00, 72.14it/s]


In [101]:
node_type_list[node_type_list.trace_id == 14000]

,trace_id,node_type,node_id
83858,14000,0,"[0, 13, 17, 22]"
83859,14000,1,"[2, 7, 11]"
83860,14000,2,"[1, 14, 18, 23]"
83861,14000,3,"[4, 8, 12]"
83862,14000,4,"[3, 9, 15, 19, 25]"
83863,14000,5,"[5, 6, 10, 16, 20, 21, 24, 26]"


In [102]:
df = pd.read_csv(f'{output_root_dir}/trace_info.csv')
df

,trace_id,trace_bool,error_trace_type,process_idx
0,0,True,normal,0
1,1,True,normal,0
2,2,True,normal,0
3,3,True,normal,0
4,4,True,normal,0
...,...,...,...,...
132480,132480,False,F04-02,8
132481,132481,False,F04-02,8
132482,132482,False,F04-02,8
132483,132483,False,F04-02,8


## Node Attributes

In [7]:
import json
from tqdm import tqdm
import pandas as pd
from collections import defaultdict

node_type_df = defaultdict(list)
for idx in range(0, 9):
    with open(f'{graph_data_path_root}/process{idx}.jsons', 'r') as fin:
        line = fin.readline()
        while line:
            trace = json.loads(line)
            node_info = trace['node_info']
            for i, vector in enumerate(node_info):
                node_type_df['trace_id'].append(trace['trace_id'])
                node_type_df['node_id'].append(i)
                node_type_df['node_type'].append(vector[4])

            line = fin.readline()
        break
    break

node_type_df = pd.DataFrame(node_type_df)
node_type_df

,trace_id,node_id,node_type
0,007efb9578bc4f1ab3eab03cb5188af5.38.1629255886...,0,0
1,007efb9578bc4f1ab3eab03cb5188af5.38.1629255886...,1,2
2,007efb9578bc4f1ab3eab03cb5188af5.38.1629255886...,2,4
3,007efb9578bc4f1ab3eab03cb5188af5.38.1629255886...,3,1
4,007efb9578bc4f1ab3eab03cb5188af5.38.1629255886...,4,1
...,...,...,...
3010748,ffc3e79734424b65942466e0d8d432e2.47.1628705856...,14,4
3010749,ffc3e79734424b65942466e0d8d432e2.47.1628705856...,15,1
3010750,ffc3e79734424b65942466e0d8d432e2.47.1628705856...,16,3
3010751,ffc3e79734424b65942466e0d8d432e2.47.1628705856...,17,4


In [22]:
edge_index_df = pd.read_csv('../../HetGNN/ProcessedData_HetGCN/edge_index.csv')
# edge_index_df['weight'] = edge_index_df['edge_type'] + 1
edge_index_df

,src_id,dst_id,edge_type,trace_id,weight
0,0,2,0,0,1
1,2,3,0,0,1
2,3,4,0,0,1
3,4,5,0,0,1
4,5,6,0,0,1
...,...,...,...,...,...
29677741,14,15,0,132484,1
29677742,15,16,0,132484,1
29677743,16,17,0,132484,1
29677744,17,18,0,132484,1


In [23]:
edge_index_df.edge_type.unique()

array([0, 1, 2, 3])